In [5]:
import pandas as pd

import tkinter as tk
from tkinter import filedialog as fd

import logging
import logging.config
import pymysql
import paramiko
from paramiko import SSHClient
from sshtunnel import SSHTunnelForwarder

import ast

C:\Users\UTEA\anaconda3\envs\utea_estimativas\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


In [6]:
import re
from unidecode import unidecode

In [7]:
from datetime import datetime

In [8]:
# ruta de carpera donde estan las llaves
ruta = open('../ruta.txt')
ruta = ruta.readlines()[0]

# cargar datos para conectar a AGROCITTCA
data = open(ruta + '/_keys/agrocittca.data','r')
password = data.readlines()[0]
data.close()

# extraccion de los datos
datos_cnx = ast.literal_eval(password)

# lectura de llave ssh
mypkey = paramiko.RSAKey.from_private_key_file(ruta + '/_keys/llave_db_agrocittca', password='micontrasena')
# ssh variables
ssh_host = datos_cnx['ssh_host']
ssh_port = datos_cnx['ssh_port']
localhost = datos_cnx['localhost']
localport = datos_cnx['localport']
ssh_user = datos_cnx['ssh_user']
# database variables
sql_username = datos_cnx['sql_username']
sql_password = datos_cnx['sql_password']
sql_main_database = datos_cnx['sql_main_database']

In [9]:
# funcion para ejecutar una script SQL
def dbQuery(q, estado = 0):
    """ access the database over the SSH tunnel and execute the query """
    logger = logging.getLogger()
    logger.debug("Connecting to reporting server...", end="")
    with SSHTunnelForwarder(
        (ssh_host, ssh_port),
        ssh_username=ssh_user,
        ssh_pkey=mypkey,
        remote_bind_address=(localhost, 3306)) as tunnel:
        logger.debug("Connected")
        logger.debug("Connecting to database...")
        try:
            conn = pymysql.connect(
                host='127.0.0.1', 
                user=sql_username,
                passwd=sql_password,
                db=sql_main_database,
                port=tunnel.local_bind_port,
                cursorclass=pymysql.cursors.DictCursor)
            logger.debug("Connected")
            logger.debug("Executing query...")
            cursor = conn.cursor()
            cursor.execute(q)
            if estado==1:
                conn.commit()  # Commit the transaction
            data = cursor.fetchall()
            logger.debug("Done")
        finally:
            conn.close()
    return data

In [10]:
# abrebiar nombre de inst
def abbreviate_institution_name(name):
    # Eliminar acentos y otros caracteres especiales
    name = unidecode(name)
    # Mantener solo caracteres alfanuméricos, espacios y guiones
    name = re.sub(r"[^\w\s-]", "", name)
    # Convertir a minúsculas
    name = name.lower()
    # Reemplazar espacios y guiones por un solo guión
    abbreviated = re.sub(r"[\s-]+", "-", name)
    return abbreviated.strip('-')

In [11]:
# open file dialog
root = tk.Tk()
root.attributes('-topmost', True)
root.iconify()
file_name = fd.askopenfilename(parent=root, filetypes=[(".xlsx", ".xlsx")])
root.destroy()

In [12]:
file_name

'D:/OneDrive - Ingenio Azucarero Guabira S.A/APP CAÑERO/DATA PARA CARGAR/INSTITUCIONES.xlsx'

In [13]:
data = pd.read_excel(file_name)

In [14]:
data.head(3)

,cod_inst,nom_inst,telf,direccion,cell,obs
0,1,1 - ASOCIACIÓN DE PEQUEÑOS PRODUCTORES DE CAÑ...,9220257,C/ Rosendo Paz Nº 148,NaN,NaN
1,3,3 - ASOCIACIÓN AGROPECUARIA 21 DE ABRIL,9221774,C/ Rosendo Paz Nº 146,NaN,NaN
2,5,5 - ASOC. DE CAÑEROS SANTISTEBAN,9220838,Av. Paichane Esq. Evaristo Mendez,NaN,NaN


In [15]:
fila = data.iloc[0]
fila['cod_inst']

1

In [16]:
# optiene la fecha actual
now = datetime.now()
formatted_now = now.strftime("%Y-%m-%d %H:%M:%S")
print(formatted_now)

2023-05-10 16:20:24


In [17]:
# seleccionar instituacion
select = 'select * from institucion where codigo=\'{inst}\''.format(inst=50)
# ejecutar seleccion
select_inst = dbQuery(select)

In [18]:
select_inst

[{'id': 2,
  'codigo': '50',
  'nombre': '50 - ASOCIACIÓN AGROPECUARIA MONTERO 1965',
  'slug': '50-asociacion-agropecuaria-montero-1965-2',
  'direccion': 'Carretera Montero Santa Cruz',
  'telefono': '9220080',
  'estado': 'activo',
  'created_at': datetime.datetime(2022, 9, 1, 7, 35, 47),
  'updated_at': datetime.datetime(2022, 9, 1, 7, 35, 47)}]

In [22]:
query = 'insert into institucion (codigo, nombre, slug, direccion, telefono, estado, created_at, updated_at) values ({codigo}, \'{nombre}\', \'{slug}\',\'{direccion}\', \'{telefono}\',\'{estado}\', \'{created_at}\', \'{updated_at}\')'.format(codigo=fila['cod_inst'], nombre=fila['nom_inst'], slug='', direccion=fila['direccion'], telefono=fila['telf'], estado='activo', created_at=formatted_now, updated_at=formatted_now)

In [23]:
query

"insert into institucion (codigo, nombre, slug, direccion, telefono, estado, created_at, updated_at) values (1, '1 -  ASOCIACIÓN DE PEQUEÑOS PRODUCTORES DE CAÑA CAMPESINO C.T.C.', '','C/ Rosendo Paz Nº 148', '9220257','activo', '2023-05-10 16:20:24', '2023-05-10 16:20:24')"

In [24]:
insert_row = dbQuery(query, 1)

In [25]:
# seleccionar instituacion
select = 'select * from institucion where codigo=\'{inst}\''.format(inst=fila['cod_inst'])
# ejecutar seleccion
select_inst = dbQuery(select)

In [26]:
inst = select_inst[0]
inst

{'id': 24,
 'codigo': '1',
 'nombre': '1 -  ASOCIACIÓN DE PEQUEÑOS PRODUCTORES DE CAÑA CAMPESINO C.T.C.',
 'slug': '',
 'direccion': 'C/ Rosendo Paz Nº 148',
 'telefono': '9220257',
 'estado': 'activo',
 'created_at': datetime.datetime(2023, 5, 10, 16, 20, 24),
 'updated_at': datetime.datetime(2023, 5, 10, 16, 20, 24)}

In [27]:
inst['nombre'] = abbreviate_institution_name(inst['nombre'])
inst

{'id': 24,
 'codigo': '1',
 'nombre': '1-asociacion-de-pequenos-productores-de-cana-campesino-ctc',
 'slug': '',
 'direccion': 'C/ Rosendo Paz Nº 148',
 'telefono': '9220257',
 'estado': 'activo',
 'created_at': datetime.datetime(2023, 5, 10, 16, 20, 24),
 'updated_at': datetime.datetime(2023, 5, 10, 16, 20, 24)}